(Adapted from https://github.com/jupyter-widgets/tutorial)

# Interactions with bqplot

In this example we will harness the power of bqplot together with ipywidgets, and see how easy it is to interact with bqplot plots.

In [1]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import *
from traitlets import *

import numpy as np
import pandas as pd
import bqplot as bq
import datetime as dt

# Every component of the figure is an independent widget

In [2]:
xs = bq.LinearScale()
ys = bq.LinearScale()
x, y = np.random.rand(2, 20)
scatt = bq.Scatter(x=x, y=y, scales={'x': xs, 'y': ys}, default_colors=['blue'])
xax = bq.Axis(scale=xs, label='x', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

fig = bq.Figure(marks=[scatt], axes=[xax, yax], animation_duration=1000)
display(fig)

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='verti…

In [11]:
fig

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', ori…

In [12]:
#data updates
scatt.x = np.random.rand(20) * 10
scatt.y = np.random.rand(20)

## The same holds for the attributes of scales, axes

In [13]:
xs.min = 4

In [14]:
xs.min = None

In [15]:
xax.label = 'Some label for the x axis'

## Use bqplot figures as input widgets

In [17]:
xs = bq.LinearScale()
ys = bq.LinearScale()
x = np.arange(100)
y = np.cumsum(np.random.randn(2, 100), axis=1) #two random walks

line = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys}, colors=['red', 'green'])
xax = bq.Axis(scale=xs, label='x', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

## Selections

In [18]:
def interval_change_callback(change):
    db.value = str(change['new'])

intsel = bq.interacts.FastIntervalSelector(scale=xs, marks=[line])
intsel.observe(interval_change_callback, names=['selected'] )

db = widgets.Label()
db.value = str(intsel.selected)
display(db)

Label(value='None')

In [19]:
fig = bq.Figure(marks=[line], axes=[xax, yax], animation_duration=1000, interaction=intsel)
display(fig)

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', ori…

In [22]:
line.selected

[46, 47, 48, 49, 50, 51, 52, 53, 54, 55]

# Hand draw

In [23]:
handdraw = bq.interacts.HandDraw(lines=line)
fig.interaction = handdraw

In [24]:
line.y[0]

array([  0.74466956,   1.13680649,   0.13805719,   0.58320537,
        -0.37561282,  -0.51530075,  -1.51399464,  -2.40070995,
        -1.90062812,  -0.57417181,   0.22417868,   0.51011892,
        -1.02001559,  -2.56754191,  -3.6727439 ,  -2.4349408 ,
        -3.07811249,  -2.59486593,  -3.08897349,  -8.21278738,
        -8.87415484,  -8.87415484,  -8.87415484,  -4.42495556,
        -9.71589525,  -9.71589525, -10.37726271, -10.49751134,
       -10.79813291, -11.09875449, -11.33925174, -11.45950037,
       -11.579749  , -11.63987332, -11.76012195, -11.94049489,
        -1.05799394,  -0.63712373,  -0.45675079,  -0.03588059,
        -0.03588059,   0.14449236,   0.50523825,   0.50523825,
         0.68561119,   0.86598413,   0.86598413,   0.98623276,
         0.98623276,   0.92610845,   0.86598413,   0.7457355 ,
         0.62548688,   0.20461667,   0.20461667,  -0.03588059,
        -0.03588059,  -0.57699942,  -0.57699942,  -0.75737236,
        -1.17824257,  -1.17824257,  -1.41873982,  -1.41

# Moving points around

In [25]:
from bqplot import *

size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)

## Enabling moving of points in scatter. Try to click and drag any of the points in the scatter and 
## notice the line representing the mean of the data update

sc_x = LinearScale()
sc_y = LinearScale()

scat = Scatter(x=x_data[:10], y=y_data[:10], scales={'x': sc_x, 'y': sc_y}, default_colors=['blue'],
               enable_move=True)
lin = Lines(scales={'x': sc_x, 'y': sc_y}, stroke_width=4, line_style='dashed', colors=['orange'])
m = Label(value='Mean is %s'%np.mean(scat.y))

def update_line(change):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]
        m.value='Mean is %s'%np.mean(scat.y)
        

update_line(None)

# update line on change of x or y of scatter
scat.observe(update_line, names='x')
scat.observe(update_line, names='y')

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')

fig = Figure(marks=[scat, lin], axes=[ax_x, ax_y])

## In this case on drag, the line updates as you move the points.
with scat.hold_sync():
    scat.enable_move = True
    scat.update_on_move = True
    scat.enable_add = False

display(m, fig)

Label(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}}, tooltip_style={'opacity': 0.9})

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='0.2f')]…